# MuSiQue single-hop baseline

In [ ]:
#|default_exp musique.singlehop

In [ ]:
#|hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#|export

from typing import Callable

import pandas as pd
from tqdm.auto import tqdm

from bellek.jerx.reward.llm import QuestionAnsweringResult
from bellek.musique.eval import calculate_metrics, compare_answers

tqdm.pandas()

In [ ]:
#|export

def make_docs(example):
    ps = example["paragraphs"]
    for p in ps:
        idx = p["idx"]
        title = p["title"]
        body = p["paragraph_text"]
        is_supporting = p["is_supporting"]
        text = f"# {title}\n{body}"
        yield dict(
            text=text,
            metadata={"parent_id": example["id"], "idx": idx, "is_supporting": is_supporting},
        )

In [ ]:
#|export

class BaselineSingleHop:
    def __init__(self, qa_func, retrieval_func):
        self.qa_func = qa_func
        self.retrieval_func = retrieval_func

    def _call(self, example) -> QuestionAnsweringResult:
        docs = list(make_docs(example))
        retrieved_docs = self.retrieval_func(docs, example['question'])
        context = "\n\n".join([doc["text"] for doc in retrieved_docs])
        return self.qa_func(context=context, question=example['question'])

    def __call__(self, example, ignore_errors: bool = False) -> QuestionAnsweringResult:
        try:
            output = self._call(example)
        except Exception as exc:
            if ignore_errors:
                id = example['id']
                print(f"Failed to answer the question {id}\n{exc}")
                output = QuestionAnsweringResult(reasoning="", answer="N/A", raw_output=str(exc))
            else:
                raise
        return output

In [ ]:
#|export

def benchmark(
    dataf: pd.DataFrame,
    qa_func: Callable,
    retrieval_func: Callable,
    only_supporting: bool = True,
    ignore_errors: bool = False,
) -> tuple[pd.DataFrame, dict]:
    pipeline = BaselineSingleHop(qa_func, retrieval_func)

    def process(example):
        output = pipeline(example, ignore_errors=ignore_errors)
        example["predicted_answer"] = output.answer
        example["raw_llm_output"] = output
        return example

    dataf = dataf.progress_apply(process, axis=1)
    dataf = compare_answers(dataf)
    scores = calculate_metrics(dataf)
    scores["fuzzy_match"] = dataf["fuzzy_match"].mean()
    return dataf, scores

In [ ]:
df = pd.read_json('../data/generated/musique-evaluation/dataset.jsonl', orient='records', lines=True)
print(len(df))
df.head()

100


,id,paragraphs,question,question_decomposition,answer,answer_aliases,answerable,answers
0,2hop__131818_161450,"[{'idx': 0, 'title': 'Maria Carrillo High Scho...",Where is the Voshmgir District located?,"[{'id': 131818, 'question': 'Which state is Vo...",in the north-east of the country south of the ...,"[Caspian Sea, in the north-east of the country...",True,"[Caspian Sea, in the north-east of the country..."
1,2hop__444265_82341,"[{'idx': 0, 'title': 'Ocala, Florida', 'paragr...",In what part of Florida is Tom Denney's birthp...,"[{'id': 444265, 'question': 'Tom Denney >> pla...",in Northern Florida,"[in Northern Florida, Northern Florida]",True,"[in Northern Florida, Northern Florida]"
2,2hop__711946_269414,"[{'idx': 0, 'title': 'Wild Thing (Tone Lōc son...",What record label is the performer who release...,"[{'id': 711946, 'question': 'All Your Faded Th...",Kill Rock Stars,[Kill Rock Stars],True,[Kill Rock Stars]
3,2hop__311931_417706,"[{'idx': 0, 'title': 'The Main Attraction (alb...",What record label does the performer of Emotio...,"[{'id': 311931, 'question': 'Emotional Rain >>...",Attic Records,"[Attic, Attic Records]",True,"[Attic, Attic Records]"
4,2hop__809785_606637,"[{'idx': 0, 'title': 'The Main Attraction (alb...",What record label does the performer of Advent...,"[{'id': 809785, 'question': 'Adventures in You...",Secret City Records,[Secret City Records],True,[Secret City Records]


In [ ]:
from bellek.jerx.reward.llm import make_question_answer_func

qa_func = make_question_answer_func()
retrieval_func = lambda docs, query: docs
pipeline = BaselineSingleHop(qa_func, retrieval_func)

In [ ]:
i = 0
example = df.iloc[i].to_dict()
output = pipeline(example).dict()
print("Question:", example['question'])
print("Reference answer:", example['answer'])
print("Predicted answer:", output['answer'])
print("Reasoning:", output['reasoning'])

Question: Where is the Voshmgir District located?
Reference answer: in the north-east of the country south of the Caspian Sea
Predicted answer: Asia
Reasoning: Voshmgir District is a district in Golestan Province. Golestan Province is located in Iran. Iran is located in Asia.


In [ ]:
mdf, scores = benchmark(df.sample(2), qa_func, retrieval_func)
print(scores)
mdf

  0%|          | 0/2 [00:00<?, ?it/s]

{'exact_match': 0.5, 'f1': 0.75, 'fuzzy_match': 0.5}


,id,paragraphs,question,question_decomposition,answer,answer_aliases,answerable,answers,predicted_answer,raw_llm_output,exact_match,fuzzy_match
18,2hop__424908_500483,"[{'idx': 0, 'title': 'Tur Dolny', 'paragraph_t...",What shares a border with the administrative d...,"[{'id': 424908, 'question': 'Bieliny, Świętokr...",Gmina Daleszyce,[Gmina Daleszyce],True,[Gmina Daleszyce],Gmina Pierzchnica.,reasoning='Bieliny is the capital of Gmina Bie...,False,False
97,2hop__739909_807845,"[{'idx': 0, 'title': 'The Journal of Life Scie...",What publisher was formed from the owner of th...,"[{'id': 739909, 'question': 'The Daily News >>...",Digital First Media,"[Digital First Media, MediaNews Group]",True,"[Digital First Media, MediaNews Group]",Digital First Media,reasoning='The Daily News is owned by MediaNew...,True,True


In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()